# Welcome to Semantic Kernel with Microsoft Fabric - Getting Started
- This notebook has following examples:
    - install semantic-kernel library in Microsoft Fabric Spark environment
    - Run prompts inline, directly in the code
    - Run prompts from a file in the LakeHouse
- Todo: Secrets are hardcoded in this notebook for simplicity for the workshop - to move to Azure Keyvault
- Prerequisites:
    - Access to Azure OpenAI, and values for the endpoint, the key, the deployments
    - Upload Skills to the Lakehouse (Todo - Write instructions for how to do this)

In [ ]:
%pip install semantic-kernel --quiet
%pip show semantic-kernel

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

In [ ]:
deployment = "text-davinci-003"    ###"gpt-35-turbo" 
api_key = "********"
endpoint = "https://******.openai.azure.com/"
#print(deployment, api_key, endpoint)

In [ ]:
kernel = sk.Kernel()
kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))


In [ ]:
# Wrap your prompt in a function
prompt = kernel.create_semantic_function("""
Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.

With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.

The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.

Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

Write a slogan or sales pitch.
""",
        max_tokens=512,
        temperature=0.0,
        top_p=0.0,
        presence_penalty=0.0,
        frequency_penalty=0.0,)

In [ ]:
# Run your prompt
print(prompt()) 

In [ ]:
prompt = """
Generate a short funny poem or limerick to explain the given event. Be creative and be funny. Let your imagination run wild.

+++++

Event:{{$input}}
+++++
"""

In [ ]:
promptFunction = kernel.create_semantic_function(
        prompt,
        max_tokens=100,
        temperature=0.9,
        top_p=0.0,
        presence_penalty=0.0,
        frequency_penalty=0.0,
    )

In [ ]:
input_text = "Friday"
completions = await promptFunction.invoke_async(input_text)

print(completions)

In [ ]:
# note: using skills from the samples folder
skills_directory = "/lakehouse/default/Files/skills"

SummarizeFunction = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")["Summarize"]



In [ ]:
SampleText = """Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.

With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.

The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.

Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

"""

In [ ]:
print(SampleText)

In [ ]:
result = SummarizeFunction(SampleText)

print(result)

# You can also invoke functions asynchronously
# result = await jokeFunction.invoke_async("time travel to dinosaur age")
# print(result)

##### End of this Notebook